In [1]:
# %% General imports
import numpy as np
import torch

# DeepMoD stuff
from multitaskpinn import DeepMoD
from multitaskpinn.model.func_approx import NN
from multitaskpinn.model.library import Library1D
from multitaskpinn.model.constraint import LeastSquares, Ridge
from multitaskpinn.model.sparse_estimators import Threshold
from multitaskpinn.training.sparsity_scheduler import TrainTestPeriodic
from multitaskpinn.utils.logger import Logger
from multitaskpinn.training.convergence import Convergence
from multitaskpinn.training import train, log_prob_students_t, train_bayes_full, train_bayes_type_II, train_SBL
from sklearn.linear_model import LassoCV

from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta
from deepymod.analysis import load_tensorboard
from deepymod.data.kdv import DoubleSoliton


%load_ext autoreload
%autoreload 2

In [24]:
# Settings
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

device='cpu'
# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Making dataset

In [34]:
# Making dataset
v = 0.25
A = 1.0

x = np.linspace(-3, 4, 40)
t = np.linspace(0.1, 5.0, 25)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=0.1, random=True, normalize=True)
X, y = X.to(device), y.to(device)

# Bayes Training function

In [37]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1, LassoCV(fit_intercept=False, cv=20)) # Sparse estimator 
constraint = LeastSquares()#Ridge(l=1e-3)# # How to constrain
model = DeepMoD(network, library, estimator, constraint, 12).to(device) # Putting it all in the model

sparsity_scheduler = TrainTestPeriodic(periodicity=25, patience=1000, delta=1e-6) # in terms of write iterations
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=2e-3) # Defining optimizer

In [38]:
train_bayes_full(model, X, y, optimizer, sparsity_scheduler, exp_ID='bayesfullpytorch_0.10', split=0.8, write_iterations=100, max_iterations=100000, delta=1e-3, patience=1e8) 

 52800  MSE: 2.42e-04  Reg: 1.62e-07  L1: 1.94e+00 

KeyboardInterrupt: 

In [6]:
model.constraint.sparsity_masks[0]

tensor([False,  True, False, False, False,  True,  True,  True,  True, False,
         True,  True])

# KDV

In [10]:
# Making data
x = np.linspace(-10, 10, 256)
t = np.linspace(0.0, 2.0, 51)

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(DoubleSoliton, c=[5.0, 1.0], x0=[-5.0, -1.0])

X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=0.0, normalize=False, random=False)

X_train, y_train = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.1, normalize=True, random=True)
X_train, y_train = X_train.to(device), y_train.to(device)

In [11]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1, LassoCV(fit_intercept=False, cv=20)) # Sparse estimator 
constraint = LeastSquares()#Ridge(l=1e-3)# # How to constrain
model = DeepMoD(network, library, estimator, constraint, 12).to(device) # Putting it all in the model

sparsity_scheduler = TrainTestPeriodic(periodicity=25, patience=1000, delta=1e-6) # in terms of write iterations
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=2e-3) # Defining optimizer

In [12]:
train_bayes_type_II(model, X_train, y_train, optimizer, sparsity_scheduler, exp_ID='kdv_type_II_long_ols_0.1', split=0.8, write_iterations=25, max_iterations=100000, delta=1e-3, patience=1e8) 

 99975  MSE: 2.89e-03  Reg: 2.49e-07  L1: 2.63e+00 Algorithm converged. Writing model to disk.
